# 1. Import Library

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [4]:
df = pd.read_csv('/datasets/games.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/games.csv'

In [ ]:
print(df.head())

# 2. Replace column names with lowercase

In [ ]:
df.columns = df.columns.str.lower() #replace column names with lowercase

In [ ]:
print(df.columns)

# 3. Convert data to necessary types

In [ ]:
# Convert 'year_of_release' to float64, allowing missing values
df['year_of_release'] = pd.to_numeric(df['year_of_release'], errors='coerce').astype('float64') 

In [ ]:
# Convert 'critic_score' to float64
df['critic_score'] = pd.to_numeric(df['critic_score'], errors='coerce').astype('float64') 

In [ ]:
# Convert 'user_score' to float, treating 'tbd' as missing (NaN)
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce').astype('float64')

In [ ]:
df.head()

# 4. Handle missing values

In [ ]:
#count how many missing values exist in each column
print(df.isna().sum()) 

In [ ]:
# Remove rows where 'name' is missing (cannot fill, only two rows)
df = df.dropna(subset=['name']) 

In [ ]:
# Fill 'genre' missing values
moda_genero = df.groupby(['platform', 'year_of_release'])['genre'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df['genre'] = df.set_index(['platform', 'year_of_release'])['genre'].fillna(moda_genero).reset_index(drop=True)

In [ ]:
# Fill 'rating' missing values
df['critic_score'] = df.groupby(['platform', 'genre'])['critic_score'].apply(lambda x: x.fillna(x.median())).reset_index(drop=True)

# Fill 'user_score' missing values
df['user_score'] = df.groupby(['platform', 'genre'])['user_score'].apply(lambda x: x.fillna(x.median())).reset_index(drop=True)

In [ ]:
#remove missing values: age rating is not always recorded, especially in old or indie games, and fill with the 'Unknown' category
df['rating'] = df['rating'].fillna('Unknown') 

In [ ]:
print(df.isna().sum()) #confirmar se ainda tem valores ausentes

* Manter o year of release com valores ausentes por enquanto

In [ ]:
df['genre'] = df['genre'].fillna('Unknown') #Preencher o restante com valor padrão

In [ ]:
df['critic_score'] = df['critic_score'].fillna(df['critic_score'].median())
df['user_score'] = df['user_score'].fillna(df['user_score'].median()) #Preencher com a mediana global

In [ ]:
#confirm if there are still missing values
print(df.isna().sum())

# 5.Calculate total sales

In [ ]:
#create a new column with the sum of sales across all regions
df['total_sales'] =  df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1) 

In [ ]:
#visualize the data to confirm
print(df[['name', 'na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales']].head()) 

# 6. Perform exploratory data analysis (EDA)

In [ ]:
#See how many games were released in different years
jogos_por_ano = df['year_of_release'].value_counts().sort_index()

In [ ]:
print(jogos_por_ano)

# 6.1 Games released per year

In [ ]:
jogos_por_ano = df['year_of_release'].value_counts().sort_index()

# Criar o gráfico
plt.figure(figsize=(14, 6))
plt.bar(jogos_por_ano.index, jogos_por_ano.values, color='skyblue')
plt.title('Number of Games Released Per Year', fontsize=14)
plt.xlabel('Year of Release')
plt.ylabel('Number of Games')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

The chart shows three distinct phases: an initial phase with few releases (1980–1993), a period of high industry activity and consolidation (1994–2011), and a recent phase with possible underrepresentation (2012–2016). For more reliable analysis, it is recommended to focus on the interval between 1995 and 2011, which shows a higher volume and diversity of data.

In [ ]:
df.head()

In [ ]:
#Remove rows without year of release
df = df.dropna(subset=["year_of_release"])
df["year_of_release"] = df["year_of_release"].astype(int)

#Ensure the total sales column exists
if "total_sales" not in df.columns:
    df["total_sales"] = df["na_sales"] + df["eu_sales"] + df["jp_sales"] + df["other_sales"]

In [ ]:
#Group by platform and get the 5 with the most total sales
vendas_totais = df.groupby("platform")["total_sales"].sum().sort_values(ascending=False)
top_plataformas = vendas_totais.head(5).index.tolist()

# Filter only these platforms
df_top = df[df["platform"].isin(top_plataformas)]

# 6.2 Analyze sales by platform

In [ ]:
#Group by year and platform
vendas_anuais = df_top.groupby(["platform", "year_of_release"])["total_sales"].sum().reset_index()

#Create the chart
plt.figure(figsize=(12, 6))
sns.lineplot(data=vendas_anuais, x="year_of_release", y="total_sales", hue="platform")
plt.title("Top 5 Platforms by Total Global Sales")
plt.xlabel("Year of Release")
plt.ylabel("Total Sales (Millions)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 6.3. Analyze of platform Emergence and Disappearance

In [ ]:
# Analysis of the emergence and disappearance of platforms
vida_plataformas = df.groupby("platform")["year_of_release"].agg(["min", "max"]).reset_index()
vida_plataformas["active_years"] = vida_plataformas["max"] - vida_plataformas["min"]

#Show platforms that disappeared (very old last year)
desaparecidas = vida_plataformas[vida_plataformas["max"] < 2015].sort_values("max", ascending=False)
print("Plataformas que deixaram de vender recentemente:")
print(desaparecidas.head(10))

Current Conclusions:

Some platforms disappear 5 to 10 years after their peak.

The average lifespan of a platform is 6 to 10 years.

Platforms that emerged in the early 2000s were still relevant until 2013–2015.

Data Period Determination:

To build a predictive model for the year 2017, the most suitable analysis period is 2000 to 2016. This range covers console generations with market behavior similar to 2017, such as the PS2, PS3, Xbox 360, and Wii. Additionally, 2016 represents the last year with available historical data before the target forecast year.

In [ ]:
# Which platforms are leading in sales?
df_relevante = df[(df["year_of_release"] >= 2000) & (df["year_of_release"] <= 2016)].copy()
df_relevante["year_of_release"] = df_relevante["year_of_release"].astype(int) #Filtrar dados no periodo relevante

In [ ]:
vendas_totais = df_relevante.groupby("platform")["total_sales"].sum().sort_values(ascending=False) #total de vendas no periodo relevante 

In [ ]:
vendas_anuais = df_relevante.groupby(["platform", "year_of_release"])["total_sales"].sum().reset_index() #tendência de crescimento ou queda: vendas por ano

# 6.4 Sales trend of the main platforms

In [ ]:
# Create a boxplot of global game sales for each platform 
plt.figure(figsize=(14, 6))
top_platforms = vendas_totais.head(5).index.tolist()
sns.lineplot(data=vendas_anuais[vendas_anuais["platform"].isin(top_platforms)],
             x="year_of_release", y="total_sales", hue="platform")
plt.title("Global Sales Distribution by Platform (2000-2016)")
plt.xlabel("Year")
plt.ylabel("Global Sales (Millions)")
plt.grid(True)
plt.tight_layout()
plt.show() 

# 6.5 Promising platforms

In [ ]:
# Promising platforms: recent sales peak
ultimos_anos = df_relevante[df_relevante["year_of_release"] >= 2014]
vendas_recentes = ultimos_anos.groupby("platform")["total_sales"].sum().sort_values(ascending=False)
print("Platforms with high recent sales volume (2014–2016):")
print(vendas_recentes.head(5))

Conclusion:

-The Wii, PS2, and Nintendo DS showed a sharp sales decline.

-PlayStation 4 (PS4) and Xbox One presented sales growth.

-PS4, Xbox One, 3DS (Nintendo), and PC (which maintains stable and continuous sales) are lucrative potential platforms.

# 6.6 Boxplot for global sales of all games, divided by platform

In [ ]:
#Remove platforms with few games
platform_counts = df["platform"].value_counts()
plataformas_validas = platform_counts[platform_counts >= 50].index
df = df[df["platform"].isin(plataformas_validas)]

In [ ]:
# Create box plot
plt.figure(figsize=(16, 8))
sns.boxplot(data=df, x="platform", y="total_sales")
plt.title("Global Sales by Plataforms (2000–2016)")
plt.xlabel("Plataforms")
plt.ylabel("Total Sales (Millions)")
plt.xticks(rotation=45)
plt.yscale("log")  # Escala logarítmica para melhor visualização
plt.tight_layout()
plt.show()

Conclusion: The chart shows that platforms like the Wii, PS3, and Xbox 360 had a few titles with very high sales (outliers), whereas other platforms sold more consistently with fewer major blockbusters. This analysis helps determine which platforms are more likely to generate higher profit per game.

In [ ]:
# Filter data for the PS3 platform and valid years
df_ps3 = df[(df["platform"] == "PS3") & 
            (df["year_of_release"] >= 2000) & 
            (df["year_of_release"] <= 2016)].copy()

In [ ]:
# Create total sales column 
df_ps3["total_sales"] = df_ps3["na_sales"] + df_ps3["eu_sales"] + df_ps3["jp_sales"] + df_ps3["other_sales"]

In [ ]:
# Remove invalid entries in ratings
df_ps3 = df_ps3[(df_ps3["user_score"] != 'tbd') & (~df_ps3["user_score"].isnull())]
df_ps3["user_score"] = df_ps3["user_score"].astype(float)
df_ps3 = df_ps3.dropna(subset=["critic_score", "total_sales"])

# 6.7 Scatter plot - Critic Score vs Sales

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.scatterplot(data=df_ps3, x="critic_score", y="total_sales")
plt.title("Critic Score vs Sales (PS3)")
plt.xlabel("Critic Score")
plt.ylabel("Total Sales (Millions)")

In [ ]:
# Scatter plot - User Score vs Sales
plt.subplot(1, 2, 2)
sns.scatterplot(data=df_ps3, x="user_score", y="total_sales")
plt.title(" User Score vs Sales (PS3)")
plt.xlabel("User Score")
plt.ylabel("Total Sales (Millions)")

plt.tight_layout()
plt.show()

# 6.8 Correlation

In [ ]:
# Correlation
cor_critic = df_ps3["critic_score"].corr(df_ps3["total_sales"])
cor_user = df_ps3["user_score"].corr(df_ps3["total_sales"])

print(f"Correlation between critic rating and sales (PS3): {cor_critic:.2f}")
print(f"Correlation between user rating and sales (PS3): {cor_user:.2f}")

Conclusion: PS3 games sold better when well-rated by critics (a correlation of 0.38), but user ratings had almost no influence on sales (a correlation of −0.03). This indicates that critic reviews have a greater commercial impact than player opinions.

# 6.9 Find games released on more than one platform

In [ ]:
jogos_multiplataforma = df["name"].value_counts()
jogos_multiplataforma = jogos_multiplataforma[jogos_multiplataforma > 1].index

In [ ]:
# Filter only these games
df_multi = df[df["name"].isin(jogos_multiplataforma)]

# 6.10 Average sales per platform for multiplatform games

In [ ]:
media_vendas = df_multi.groupby("platform")["total_sales"].mean().sort_values(ascending=False)

print("Average sales of multiplatform games by platform:")
print(media_vendas)

# Sales of the same game on different platforms
exemplo_jogo = "Call of Duty: Black Ops"  
df_exemplo = df_multi[df_multi["name"] == exemplo_jogo][["name", "platform", "total_sales"]]
print(f"\nSales of the game'{exemplo_jogo}' on different platforms:")
print(df_exemplo)

In [ ]:
# Game count by genre
contagem_generos = df["genre"].value_counts()


In [ ]:
# Total sales by genre
vendas_genero_total = df.groupby("genre")["total_sales"].sum().sort_values(ascending=False)

In [ ]:
# Average sales by genre
vendas_genero_media = df.groupby("genre")["total_sales"].mean().sort_values(ascending=False)


# 6.11 General distribution plot of games by genre

In [ ]:
#Plot general distribution of games by genre
plt.figure(figsize=(14, 4))
sns.barplot(x=contagem_generos.index, y=contagem_generos.values)
plt.title("Distribution of Games by Genre (2000–2016)")
plt.xlabel("Genre")
plt.ylabel("Game Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 6.12 Total sales by genre plot

In [ ]:
#Plot total sales by genre
plt.figure(figsize=(14, 4))
sns.barplot(x=vendas_genero_total.index, y=vendas_genero_total.values)
plt.title("Total Sales by Genre (2000–2016)")
plt.xlabel("Genre")
plt.ylabel("Total Sales (Millions)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 6.13 Average sales per game in each genre plot

In [ ]:
#Plot average sales per game in each genre
plt.figure(figsize=(14, 4))
sns.barplot(x=vendas_genero_media.index, y=vendas_genero_media.values)
plt.title("Average Sales by Genre (2000–2016)")
plt.xlabel("Genre")
plt.ylabel("Average Sales per Game")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Conclusion: The charts show that the most frequently released genres are Action and Sports. However, the most lucrative per game are Shooter and Platform, with the highest average sales. Conversely, genres like Puzzle, Adventure, and Strategy sell less and are therefore less profitable per title.

# 7. Create a user profile for each region

In [ ]:
# Filter relevant period
df = df[(df["year_of_release"] >= 2000) & (df["year_of_release"] <= 2016)]

In [ ]:
# Create a ranking of the top 5 best-selling platforms by region
top_na = df.groupby("platform")["na_sales"].sum().sort_values(ascending=False).head(5)
top_eu = df.groupby("platform")["eu_sales"].sum().sort_values(ascending=False).head(5)
top_jp = df.groupby("platform")["jp_sales"].sum().sort_values(ascending=False).head(5)


In [ ]:
# Combine the unique platforms from the top 5 in each region
plataformas_unicas = set(top_na.index).union(top_eu.index).union(top_jp.index)


In [ ]:
# Create DataFrame with sales by region for these platforms
df_plataformas = df[df["platform"].isin(plataformas_unicas)]
vendas_regiao = df_plataformas.groupby("platform")[["na_sales", "eu_sales", "jp_sales"]].sum()


In [ ]:
# Calculate the market share (%) of each platform in each region.
vendas_percentuais = vendas_regiao.div(vendas_regiao.sum(axis=0), axis=1) * 100

In [ ]:
# Sort by total sales in NA as a reference.
vendas_percentuais = vendas_percentuais.loc[list(plataformas_unicas)]
vendas_percentuais.round(1)

Conclusion: In North America and Europe, the Xbox 360 and PS3 lead in sales, reflecting a preference for home consoles. In Japan, however, handheld platforms such as the DS and PSP dominate the market. These variations demonstrate that the consumer profile for gaming changes significantly by region: traditional consoles hold more strength in the West, while Japan favors portables.

In [ ]:
# Calculate the top five genres by region (NA, EU, JP)
generos_na = df.groupby("genre")["na_sales"].sum().sort_values(ascending=False).head(5)
generos_eu = df.groupby("genre")["eu_sales"].sum().sort_values(ascending=False).head(5)
generos_jp = df.groupby("genre")["jp_sales"].sum().sort_values(ascending=False).head(5)

# Display the results
print("Top Genres by Region:")
print("\n North America (NA):")
print(generos_na)
print("\nEurope (EU):")
print(generos_eu)
print("\nJapan (JP):")
print(generos_jp)


## Regional Market Profile Summary

### Platform and Console Preferences
The market profile is starkly divided by region:
* **North America (NA) and Europe (EU)** prefer **home consoles** (Xbox 360, PS3), which dominate sales.
* **Japan (JP)** prefers **handheld platforms** (DS, PSP), which are the top sellers.
This highlights a key difference in consumer behavior: the West favors traditional consoles, while Japan prioritizes portability.

### Genre Preferences
* **NA and EU** are dominated by high-action and competitive genres: **Action, Shooter, and Sports**.
* **Japan** is dominated by immersive, story-driven genres: **Role-Playing Games (RPGs)** and **Adventure**.

### Rating Impact
* **Critic Ratings** (e.g., Metacritic) show a **strong positive correlation** with sales on the PS3, indicating that critical reviews have a clear commercial impact.
* **Age Ratings** (ESRB, PEGI, CERO) have the **highest commercial impact in NA**, where Mature-rated games face more retail friction compared to Europe. High CERO ratings in Japan can also limit title availability.

# 8. Hypothesis Testing

In [ ]:
# Perform the t-test for Xbox One and PC
# Filter data
df_xbox_pc = df[df["platform"].isin(["XOne", "PC"])]
df_xbox_pc.head()

In [ ]:
# Clean user ratings data (remove missing values)
df_xbox_pc = df_xbox_pc.dropna(subset=["user_score"])

In [ ]:
# Separate the data for the two platforms
xbox_one_scores = df_xbox_pc[df_xbox_pc["platform"] == "XOne"]["user_score"]
pc_scores = df_xbox_pc[df_xbox_pc["platform"] == "PC"]["user_score"]


In [ ]:
# Perform the two-sample independent t-test
t_stat, p_value = stats.ttest_ind(xbox_one_scores, pc_scores)

print(f"Estatística t: {t_stat:.3f}")
print(f"Valor p: {p_value:.3f}")

In [ ]:
# Test the hypothesis
alpha = 0.05
if p_value < alpha:
    print("We reject the null hypothesis: the average user ratings for Xbox One and PC are different.")
else:
    print("We fail to reject the null hypothesis: there is not enough evidence to state that the average user ratings for Xbox One and PC are different.")

In [ ]:
# Perform the t-test for the Action and Sports genres
# Filter data
df_action_sports = df[df["genre"].isin(["Action", "Sports"])]

In [ ]:
# Clean user ratings data (remove missing values)
df_action_sports = df_action_sports.dropna(subset=["user_score"])

In [ ]:
# Separate the data for the two genres
action_scores = df_action_sports[df_action_sports["genre"] == "Action"]["user_score"]
sports_scores = df_action_sports[df_action_sports["genre"] == "Sports"]["user_score"]

In [ ]:
# Perform the two-sample independent t-test
t_stat, p_value = stats.ttest_ind(action_scores, sports_scores)

print(f"Estatística t: {t_stat:.3f}")
print(f"Valor p: {p_value:.3f}")


In [ ]:
# Test the hypothesis
alpha = 0.05
if p_value < alpha:
    print("We reject the null hypothesis: the average user ratings for the Action and Sports genres are different.")
else:
    print("We fail to reject the null hypothesis: there is not enough evidence to state that the average user ratings for the Action and Sports genres are different.")

# 9. Final Conclusion

## General Project Conclusion

This project involved a statistical analysis of video game sales data to map consumer behavior and market preferences across different platforms and genres.

### Key Findings from Hypothesis Testing

We used a **Two-Sample Independent t-test** to compare average user ratings. The results showed **rejection of the null hypothesis** in both cases, confirming that the observed differences are statistically significant:

1.  **Platform Comparison (Xbox One vs. PC):** The average user ratings are **significantly different**, suggesting that user experiences and expectations for games vary between these two major platforms.
2.  **Genre Comparison (Action vs. Sports):** The average user ratings for these two popular genres are also **significantly different**, indicating clear distinctions in player satisfaction and preference.

### Descriptive Analysis and Market Trends

The market analysis revealed strong regional and genre-based trends:

* **Genre Popularity:** While **Action** and **Sports** lead in overall sales volume, specific regional tastes drive different market demands.
* **Regional Preferences:** **Action** and **Shooter** games dominate in the **Americas and Europe**. Conversely, **Role-Playing Games (RPGs)** and **Adventure** titles show a powerful preference in **Japan**.

### Conclusion on Ratings Impact

Our analysis suggests that **critic ratings tend to have a more significant influence on sales** in some platforms and genres than user ratings.

### Final Conclusion

Gaming preferences vary widely by platform and genre, with **region playing a crucial role** (RPGs in Japan vs. Action in the West). The statistically significant differences in user ratings and the varied impact of critic ratings highlight the necessity of developing and marketing products tailored to these distinct segments.